We work out by hand example 6.8 in Equivariant Ehrhart Theory by Alan Stapledon.
https://arxiv.org/abs/1003.5875

Let $G = \mathbb{Z}/2\mathbb{Z}$ with generator $\tau$, and let $\chi: G \rightarrow \mathbb C$ be the linear character sending $\tau$ to -1, so that the irreducible representations of $G$ are $1,\chi$. Let $P$ be the convex hull of (0,0),(3,0),(0,3), and (3,3), and consider the action of $G$ on $M = \mathbb Z ^2$ given by 
$$\tau \rightarrow \begin{pmatrix} -1 & 0 \\ 0 & 1 \end{pmatrix}.$$ Then, with the notation of Section 4, $P$ is $G$-invariant `up to translation,' and one computes that $\phi_2 = 5 - \chi$. Observe that in this example $\phi[t]$ is not a polynomial by Lemma 7.3.

I define the square and the automorphism group, and I run the file eetheory:


In [36]:
P = Polyhedron(vertices = [[0,0],[3,0],[0,3],[3,3]],backend = 'normaliz')
AutP = P.restricted_automorphism_group(output = 'permutation')
%runfile eetheory.py


We need to lift square so that the action of $G$ on $P$ is linear. Following section 4, the lifted action of $\tau$ is given by $\tau \cdot (u,\lambda) = (\tau \cdot u + \lambda(3,0), \lambda)$: $$\tau = 
\begin{pmatrix} 
-1 & 0 & 3 \\
 0 & 1 & 0 \\
 0 & 0 & 1 
\end{pmatrix}.$$ One can check that $\tau$ now permutes the lifted vertices. The next thing to do is create the correct subgroup of the automorphism group.

In [30]:
# Look at the dictionary from the function match_perms_to_mats to find the correct matrix. Create the subgroup.
group_dict = match_perms_to_mats(P,AutP)
print group_dict.values()[2]
print group_dict.keys()[2]
G = AutP.subgroup(gens=[group_dict.keys()[2]])
print G


[-1  0  3]
[ 0  1  0]
[ 0  0  1]
(0,2)(1,3)
Subgroup of (Permutation Group with generators [(1,2), (0,1)(2,3), (0,3)]) generated by [(0,2)(1,3)]


Calculate by hand the Ehrhart series of the fixed subpolytopes. 
For the identity element, the Ehrhart series is that of the whole square. Calculate the h* polynomial by creating a regular unimodular triangulation of the square and taking a half-open decompostion. 
$$ \frac{h^*(t)}{(1-t)^3} = \frac{1+13t+4t^2}{(1-t)^3} .$$
For the line segment [(3/2,0,1),(3/2,3,1)] fixed by $\tau$, we write the generating series for the cone with primitive ray generators (3,0,2) and (3,6,2). Then we count the number of lattice points in the fundamental parallelepiped. 

In [32]:
fund = Polyhedron(vertices = [[0,0,0],[3,0,2],[3,6,2],[6,6,4]])
fund.integral_points()

((0, 0, 0),
 (3, 0, 2),
 (3, 1, 2),
 (3, 2, 2),
 (3, 3, 2),
 (3, 4, 2),
 (3, 5, 2),
 (3, 6, 2),
 (6, 6, 4))

Now we can read off that the $h^*$ Ehrhart series of the line segment is 
$$\frac{1+5t^2}{(1-t^2)^2}.$$ Time to check if the code does the right thing. I have added several optional inputs to calculate_phi in eetheory to change the output. So there is an option to get the Ehrhart series for all fixed subpolytopes. 

In [38]:
ehr_series = calculate_phi(P,G,output = 'e_series_list')
ehr_series

[(4*t^2 + 13*t + 1)/(-t^3 + 3*t^2 - 3*t + 1), (5*t^2 + 1)/(t^4 - 2*t^2 + 1)]

So far so good. The hand calculate ehrhart series agree. Now we calculate the determinants $det(Id - \rho(g)[t])$. The determinant for the identity element of $G$ will be $(1-t)^3$. The determinant for $\tau$ will be the determinant of:
$$
\begin{pmatrix}
1+t & 0   & -3t \\
0   & 1-t & 0   \\
0   & 0   & 1-t
\end{pmatrix},
$$ which is $(1+t)(1-t)^2 = t^3 - t^2 -t +1$. Again, we check the output of the code.

In [40]:
determinant_vector = calculate_phi(P,G,output = 'determinant_vec')
determinant_vector

[-t^3 + 3*t^2 - 3*t + 1, t^3 - t^2 - t + 1]

The next step is to make sure that the multiplication between the Ehrhart series and the determinant vector is being done correctly. By hand, one checks that the product should be $4t^2 +13t +1$ and $\frac{1+5t^2}{1+t}$.

In [42]:
product = calculate_phi(P,G,output = 'prod_det_es')
product

(4*t^2 + 13*t + 1, (5*t^2 + 1)/(t + 1))

The only step that's left is to left solve in the character table to figure out phi. I won't write down the arithmetic here, but I calculate the same thing that the code gives me:

In [44]:
phi = calculate_phi(P,G)
print phi
print G.character_table()

((2*chi_0 + 2*chi_1)*t^3 + (11*chi_0 + 6*chi_1)*t^2 + (7*chi_0 + 7*chi_1)*t + chi_0)/(t + 1)
[ 1  1]
[ 1 -1]


The problem is that Stapledon says, "one computes that $\phi_2 = 5 -\chi$", but I have computed that $\phi_2 = 11 +6\chi$.

$\tt{What's \,wrong?}$

Let me convince you that the above expression of phi is correct. 
If we plug in $\chi_0 (id) = 1 $ and $\chi_1 (id) = 1$, then we get
$$\frac{4t^3 + 17t^2 +14t +1}{t+1} = 4t^2 +13t+1  ,$$ which we have already shown is the $h^*$ polynomial of the entire square. 

If we plug in $\chi_0 (\tau) = 1 $ and $\chi_1 (\tau) = -1$, then we get 
$$\frac{5t^2 +1}{t+1}.$$ 

Dividing this by $det(Id -\rho(\tau)t) = (1+t)(1-t)^2$ yields $\frac{5t^2 +1}{(1-t^2)^2}$, which is the correct ehrhart series of the fixed line segment. 